<a href="https://colab.research.google.com/github/ithingv/nlp_study/blob/main/LanguageModel_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformersのインストール
!git clone https://github.com/huggingface/transformers.git
!cd transformers/; pip install -e .

In [ ]:
import codecs
import re
import requests
from bs4 import BeautifulSoup

# URLとキーワードリスト
url = "https://ja.wikipedia.org/wiki/"
keyword_list = [
    "中島敦", "太宰治", "国木田独歩",
    "江戸川乱歩", "谷崎潤一郎", "宮沢賢治",
    "与謝野晶子", "芥川龍之介", "樋口一葉",
    "中原中也", "尾崎紅葉", "梶井基次郎",
    "夢野久作", "森鷗外", "織田作之助"   
]

# Wikipediaの小説家の記事のダウンロード
corpus = []
for keyword in keyword_list:
    response = requests.get(url + keyword)
  
    soup = BeautifulSoup(response.text, 'lxml')
    for p_tag in soup.find_all('p'):
        # 空白の削除
        text = "".join(p_tag.text.strip().split(" "))

        # 空行は無処理
        if len(text) == 0:
            continue

        # 注釈の削除 (例: [注釈1], [注釈1], [1])
        text = re.sub(r"\[注釈[0-9]+\]", "", text)
        text = re.sub(r"\[注[0-9]+\]", "", text)
        text = re.sub(r"\[[0-9]+\]", "", text)

        # 行の追加
        corpus.append(text)

# ファイルの保存       
print(*corpus, sep="\n", file=codecs.open("wiki.txt", "w", "utf-8"))

In [ ]:
# データセットの分割
!head -n100 wiki.txt > eval.txt
!tail -n+101 wiki.txt > train.txt
!wc -ml *.txt

In [ ]:
# 学習
# run_language_modeling.py
!python transformers/examples/legacy/run_language_modeling.py \
    --output_dir=output \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=eval.txt \
    --per_device_train_batch_size=2 \
    --per_device_eval_batch_size=2 \
    --num_train_epochs=10 \
    --save_steps=5000 \
    --save_total_limit=3

In [ ]:
!python transformers/examples/pytorch/text-generation/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path=output \
    --prompt "山田太郎。日本の小説家。 " \
    --seed=${RANDOM} \
    --length 1000